In [1]:
!pip install rplidar pyserial
!apt-get install -y python3-smbus
!pip install jetson-stats
!pip install adafruit-circuitpython-motorkit
!python3 setup.py install


  Preparing metadata (setup.py) ... done
  Created wheel for rplidar: filename=rplidar-0.9.2-py3-none-any.whl size=5409 sha256=e3988823984f14b2ebb5f3550bdb93afefa5f4125f2f4d04da7634d26704527b
  Stored in directory: /root/.cache/pip/wheels/98/c6/3a/77b0c32cd1a920209ecc36f20af579589fe077015f16b2c899
Successfully built rplidar


In [2]:
import time
import numpy as np
from rplidar import RPLidar
from jetracer.nvidia_racecar import NvidiaRacecar

# Parameters
MIN_DISTANCE = 1000  # Minimum distance for obstacle detection in millimeters
ANGULAR_RANGE = 30  # Angular range for obstacle detection in degrees

# Set up the RPLidar
lidar = RPLidar('/dev/ttyUSB0')
lidar.start_motor()

# Set up the JetRacer
car = NvidiaRacecar()

def detect_obstacle(scan):
    for _, angle, distance in scan:
        if distance < MIN_DISTANCE and 90 - ANGULAR_RANGE / 2 < angle < 90 + ANGULAR_RANGE / 2:
            return True
    return False

try:
    for scan in lidar.iter_scans():
        obstacle = detect_obstacle(scan)

        if obstacle:
            car.throttle = -0.2  # Reverse
            car.steering = 0.5   # Turn right
        else:
            car.throttle = 0.3  # Forward
            car.steering = 0    # Straight

        time.sleep(0.1)

except KeyboardInterrupt:
    pass

finally:
    car.throttle = 0
    car.steering = 0
    lidar.stop_motor()
    lidar.disconnect()


ModuleNotFoundError: No module named 'jetracer'